In [18]:
import sqlite3
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [19]:
##creating the database connection
conn = psycopg2.connect(
    host='localhost',
    port=5432,
    database='crop_production',
    user='postgres',
    password='aryan'
)

In [20]:
tables = pd.read_sql_query("SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'public';", conn)
tables

,tablename
0,area_codes
1,crop_production
2,elements
3,itemcodes


In [25]:
## Looking at basic information from the respective tables
for table in tables['tablename']:
    try:
        print('-' * 40, f'{table}', '-' * 40)
        query = f'SELECT COUNT(*) AS cnt FROM "{table}";'
        result = pd.read_sql_query(query, conn)
        print(pd.read_sql(f'select * from "{table}" limit 5;', conn))

        count_value = result["cnt"].iloc[0]

        print(f'Table: {table}, Count: {count_value}')
        print()
        print()
    except Exception as e:
        print(f'Table: {table}, Error: {e}')

---------------------------------------- area_codes ----------------------------------------
   Area Code M49 Code         Area
0          2     '004  Afghanistan
1       5100     '002       Africa
2          3     '008      Albania
3          4     '012      Algeria
4       5200     '019     Americas
Table: area_codes, Count: 245


---------------------------------------- crop_production ----------------------------------------
   Area Code Area Code (M49)         Area  Item Code Item Code (CPC)  \
0          2            '004  Afghanistan        221          '01371   
1          2            '004  Afghanistan        221          '01371   
2          2            '004  Afghanistan        221          '01371   
3          2            '004  Afghanistan        711          '01654   
4          2            '004  Afghanistan        711          '01654   

                                                Item  Element Code  \
0                                  Almonds, in shell          53

In [26]:
display(tables['tablename'])

0         area_codes
1    crop_production
2           elements
3          itemcodes
Name: tablename, dtype: object

In [34]:
area_codes_df = pd.read_sql_query(
    'SELECT * FROM "area_codes";',
    conn
)
area_codes_df


,Area Code,M49 Code,Area
0,2,'004,Afghanistan
1,5100,'002,Africa
2,3,'008,Albania
3,4,'012,Algeria
4,5200,'019,Americas
...,...,...,...
240,5000,'001,World
241,249,'887,Yemen
242,248,'890,Yugoslav SFR
243,251,'894,Zambia


In [35]:
##checking the uniqueness of area_code column
area_codes_df[' Area Code'].nunique() == area_codes_df.shape[0]

True

The uniqueness of the output indicates its readiness for use in additional table-aggregation steps.

In [38]:
##checking the uniqueness of item_codes column
item_codes_df = pd.read_sql_query(
    'SELECT * FROM "area_codes";',
    conn
)
item_codes_df



,Area Code,M49 Code,Area
0,2,'004,Afghanistan
1,5100,'002,Africa
2,3,'008,Albania
3,4,'012,Algeria
4,5200,'019,Americas
...,...,...,...
240,5000,'001,World
241,249,'887,Yemen
242,248,'890,Yugoslav SFR
243,251,'894,Zambia


In [39]:

##checking the uniqueness of item_codes column
item_codes_df[' Area Code'].nunique() == item_codes_df.shape[0]

True

We now look at our crop_production table to get an insight over it

In [79]:
crop_production_df = pd.read_sql_query(
    'SELECT * FROM "crop_production";',
    conn
)
crop_production_df


,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,...,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,Y2023
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,0.0,...,13703.00,14676.00,19481.00,19793.00,20053.00,29203.00,22134.00,36862.0,36462.00,37000.00
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5412,Yield,kg/ha,NaN,...,1999.60,1652.10,1685.90,1378.80,1716.10,1308.30,1775.90,1743.2,1742.00,1810.80
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,t,0.0,...,27400.00,24246.00,32843.00,27291.00,34413.00,38205.00,39307.00,64256.0,63515.00,67000.00
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,NaN,...,30000.00,25000.00,24500.00,26160.00,25220.00,25293.00,25558.00,25357.0,25403.00,25439.00
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5412,Yield,kg/ha,NaN,...,716.70,720.00,707.50,697.00,708.00,704.10,703.00,705.0,704.00,704.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78159,5817,'902,Net Food Importing Developing Countries (NFIDCs),1729,'F1729,"Treenuts, Total",5412,Yield,kg/ha,614.5,...,531.80,542.20,574.40,592.70,536.40,654.20,636.70,683.0,676.60,682.70
78160,5817,'902,Net Food Importing Developing Countries (NFIDCs),1729,'F1729,"Treenuts, Total",5510,Production,t,255724.0,...,1877336.94,2189212.78,2022188.76,2299639.12,2494153.15,2313287.62,2480759.06,2667688.9,2854981.94,2812938.79
78161,5817,'902,Net Food Importing Developing Countries (NFIDCs),1735,'F1735,Vegetables Primary,5312,Area harvested,ha,1881425.0,...,6981369.00,7281896.00,7311844.00,7258558.00,7462627.00,7520604.00,7692944.00,7753692.0,7754677.00,7795058.00
78162,5817,'902,Net Food Importing Developing Countries (NFIDCs),1735,'F1735,Vegetables Primary,5412,Yield,kg/ha,6617.1,...,11684.90,11691.50,11615.90,11781.10,11747.00,11880.00,11807.20,12136.8,12442.00,12396.10


In [80]:
year_cols = [c for c in crop_production_df.columns if c.startswith("Y")]

df_long = crop_production_df.melt(
    id_vars=[c for c in crop_production_df.columns if c not in year_cols],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value"
)

df_long["Year"] = df_long["Year"].str[1:].astype(int)

df_long.head()


,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Year,Value
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,1961,0.0
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5412,Yield,kg/ha,1961,NaN
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,t,1961,0.0
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,1961,NaN
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5412,Yield,kg/ha,1961,NaN


We can notice that the production table is in wide format(horizontally) which means that we need to convert it into long(vertical) format for ease

In [82]:
df_long.tail(10)

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Year,Value
4924322,5817,'902,Net Food Importing Developing Countries (NFIDCs),1809,'F1809,"Skim Milk & Buttermilk, Dry",5510,Production,t,2023,NaN
4924323,5817,'902,Net Food Importing Developing Countries (NFIDCs),1723,'F1723,Sugar Crops Primary,5312,Area harvested,ha,2023,3.510373e+06
4924324,5817,'902,Net Food Importing Developing Countries (NFIDCs),1723,'F1723,Sugar Crops Primary,5412,Yield,kg/ha,2023,6.571750e+04
4924325,5817,'902,Net Food Importing Developing Countries (NFIDCs),1723,'F1723,Sugar Crops Primary,5510,Production,t,2023,2.306929e+08
4924326,5817,'902,Net Food Importing Developing Countries (NFIDCs),1729,'F1729,"Treenuts, Total",5312,Area harvested,ha,2023,4.120104e+06
4924327,5817,'902,Net Food Importing Developing Countries (NFIDCs),1729,'F1729,"Treenuts, Total",5412,Yield,kg/ha,2023,6.827000e+02
4924328,5817,'902,Net Food Importing Developing Countries (NFIDCs),1729,'F1729,"Treenuts, Total",5510,Production,t,2023,2.812939e+06
4924329,5817,'902,Net Food Importing Developing Countries (NFIDCs),1735,'F1735,Vegetables Primary,5312,Area harvested,ha,2023,7.795058e+06
4924330,5817,'902,Net Food Importing Developing Countries (NFIDCs),1735,'F1735,Vegetables Primary,5412,Yield,kg/ha,2023,1.239610e+04
4924331,5817,'902,Net Food Importing Developing Countries (NFIDCs),1735,'F1735,Vegetables Primary,5510,Production,t,2023,9.662829e+07


In [102]:
##checking for missing values in each year column
crop_production_df[year_cols].isna().sum().sort_values()


Y2017     8207
Y2022     8409
Y2016     8431
Y2015     8518
Y2021     8541
         ...  
Y1964    22049
Y1965    22054
Y1962    22063
Y1963    22064
Y1961    22070
Length: 63, dtype: int64

In [73]:
# 1. Identify year columns safely
year_cols = [
    c for c in crop_production_df.columns
    if c.startswith("Y") and c[1:].isdigit()
]

print("Number of year columns:", len(year_cols))
print("First 5:", year_cols[:5])
print("Last 5:", year_cols[-5:])


# 2. Melt wide → long
df_long = crop_production_df.melt(
    id_vars=[c for c in crop_production_df.columns if c not in year_cols],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value"
)


# 3. Clean year ("Y1961" → 1961)
df_long["Year"] = df_long["Year"].str[1:].astype(int)

# 4. View result
df_long.head()


Number of year columns: 63
First 5: ['Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965']
Last 5: ['Y2019', 'Y2020', 'Y2021', 'Y2022', 'Y2023']


,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Year,Value
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,1961,0.0
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5412,Yield,kg/ha,1961,NaN
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,t,1961,0.0
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,1961,NaN
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5412,Yield,kg/ha,1961,NaN


In [83]:
df_long.head()
df_long.shape
df_long.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4924332 entries, 0 to 4924331
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Area Code        int64  
 1   Area Code (M49)  object 
 2   Area             object 
 3   Item Code        int64  
 4   Item Code (CPC)  object 
 5   Item             object 
 6   Element Code     int64  
 7   Element          object 
 8   Unit             object 
 9   Year             int32  
 10  Value            float64
dtypes: float64(1), int32(1), int64(3), object(6)
memory usage: 394.5+ MB


We have successfully converted the data into an improved format, which will allow us to work with it more effectively.


In [ ]:
##making sure the data types are correct
df_long["Year"] = df_long["Year"].astype(int)
df_long["Value"] = pd.to_numeric(df_long["Value"], errors="coerce")


In [87]:
df_long.head(20)

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Year,Value
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,ha,1961,0.0
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5412,Yield,kg/ha,1961,NaN
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,t,1961,0.0
3,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5312,Area harvested,ha,1961,NaN
4,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5412,Yield,kg/ha,1961,NaN
5,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5510,Production,t,1961,NaN
6,2,'004,Afghanistan,515,'01341,Apples,5312,Area harvested,ha,1961,2220.0
7,2,'004,Afghanistan,515,'01341,Apples,5412,Yield,kg/ha,1961,6801.8
8,2,'004,Afghanistan,515,'01341,Apples,5510,Production,t,1961,15100.0
9,2,'004,Afghanistan,526,'01343,Apricots,5312,Area harvested,ha,1961,4820.0


In [ ]:
##renaming the columns for ease of use
df_long = df_long.rename(columns={
    "Area Code": "area_code",
    "Area Code (M49)": "area_m49",
    "Area": "area_name",
    "Item Code": "item_code",
    "Item Code (CPC)": "item_cpc",
    "Item": "item_name",
    "Element Code": "element_code",
    "Element": "element_name",
    "Unit": "unit",
    "Year": "year",
    "Value": "value"
})

In [89]:
##segregating thee data we are interested in

focus_elements = ["Production", "Area harvested", "Yield"]
df_focus = df_long[df_long["element_name"].isin(focus_elements)]


In [ ]:
##creating a pivot table to have separate columns for each of the focus elements
metrics_df = df_focus.pivot_table(
    index=["area_code", "area_name", "item_code", "item_name", "year"],
    columns="element_name",
    values="value",
    aggfunc="first"
).reset_index()


In [91]:
metrics_df.columns.name = None

metrics_df = metrics_df.rename(columns={
    "Production": "prod_tonnes",
    "Area harvested": "area_ha",
    "Yield": "yield_kg_ha"
})


In [96]:
metrics_df.head()

,area_code,area_name,item_code,item_name,year,area_ha,prod_tonnes,yield_kg_ha
0,1,Armenia,15,Wheat,1992,65500.0,141483.0,2160.0
1,1,Armenia,15,Wheat,1993,97900.0,217900.0,2225.7
2,1,Armenia,15,Wheat,1994,85697.0,152900.0,1784.2
3,1,Armenia,15,Wheat,1995,60362.0,153800.0,2548.0
4,1,Armenia,15,Wheat,1996,85450.0,201400.0,2356.9


In [98]:
metrics_df.shape


(1555222, 8)

In [ ]:
#saving the long (vertical) version of the csv
df_long.to_csv("faostat_long.csv", index=False)


In [105]:
import os
print(f'File size: {os.path.getsize("data/faostat_long.csv") / 1024**2:.2f} MB')  


File size: 420.65 MB


In [101]:
metrics_df.to_csv("faostat_crop_year_metrics.csv", index=False)
